In [9]:
  ! pip install biopython

Obtain the raw data

In [10]:
! wget 'https://github.com/sonluongvu/Palm_structure/raw/main/Homo_sapiens_info.csv'

--2022-01-22 20:50:53--  https://github.com/sonluongvu/Palm_structure/raw/main/Homo_sapiens_info.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/Homo_sapiens_info.csv [following]
--2022-01-22 20:50:54--  https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/Homo_sapiens_info.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 619542 (605K) [text/plain]
Saving to: ‘Homo_sapiens_info.csv.1’

Homo_sapiens_info.c 100%[===================>] 605.02K  --.-KB/s    in 0.007s  

2022-01-22 20:50:54 (82.7 MB/s) - ‘Homo_sapiens_info.csv.1’ saved [619542/619542]


Open the raw data

In [11]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio.Align import AlignInfo
from Bio import AlignIO
from Bio.Align.Applications import MuscleCommandline

In [12]:
df_path = '/content/Homo_sapiens_info.csv'
df = pd.read_csv(df_path)
df.shape

(643, 4)

Import libraries

Extract peptides 100 aa upstream and downstream of palmitoylated C

In [13]:
peptide_record = []
upstream_peptide_record = []
downstream_peptide_record = []

for index, row in df.iterrows():
  pos = row['pos']
  start = pos - 1 + 200 - 100
  end = pos - 1 + 200 + 101

  original_seq = row['protein_seq']
  seq_id = row['id']
  
  peptide_sequence = Seq(original_seq[start:end])
  upstream_seq = Seq(original_seq[start:(pos-1+200)])
  downstream_seq = Seq(original_seq[(pos+200):end])

  record = SeqRecord(peptide_sequence, id = seq_id)
  upstream_record = SeqRecord(upstream_seq, id = seq_id)
  downstream_record = SeqRecord(downstream_seq, id = seq_id)

  peptide_record.append(record)
  upstream_peptide_record.append(upstream_record)
  downstream_peptide_record.append(downstream_record)

Create the sequence alginment

In [14]:
alignment = MultipleSeqAlignment(peptide_record)
AlignIO.write(alignment, 'alignments.fasta','fasta')

upstream_alignment = MultipleSeqAlignment(upstream_peptide_record)
AlignIO.write(upstream_alignment, 'upstream_alignments.fasta','fasta')

downstream_alignment = MultipleSeqAlignment(downstream_peptide_record)
AlignIO.write(downstream_alignment, 'downstream_alignments.fasta','fasta')

1

In [15]:
alignment_info = AlignInfo.SummaryInfo(alignment)
alignment_pssm = alignment_info.pos_specific_score_matrix()

Create dataframe from the pssm

In [16]:
aa_list = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
pssm_df = pd.DataFrame()
for aa in aa_list:
  column_value =[]
  for row in range(201):
    value = alignment_pssm[row][aa]
    column_value.append(value)
  pssm_df[aa] = column_value
pssm_df.to_csv('pssm.csv', index = False)